In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime,timedelta
import os
from fastprogress.fastprogress import master_bar, progress_bar
pd.set_option('mode.chained_assignment','raise')

/home/lulu/.pyenv/versions/3.9.7/envs/study_1_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
directory_pc_per_city = '../data/processed_data/zones_delineation/'

In [3]:
zones = gpd.GeoDataFrame({'Postcode':[],'gemeente':[],'id_unit':[],'area_ratio':[],'geometry':[]}, 
                         crs = 'EPSG:28992')

In [4]:
list_files = os.listdir(directory_pc_per_city)
list_files.remove('edited')
pg = progress_bar(range(len(list_files)))
for i in pg:
    file = list_files[i]
    if file in ('Buitenland.gpkg','README_zones_delineation.mkd'):
        continue
    # Removing the extension.
    city_name = file[3:-5]
    pg.comment = city_name
    
    if os.path.isfile(directory_pc_per_city + 'edited/' + file):
        zones_city = gpd.read_file(directory_pc_per_city + 'edited/' +  file)
        zones = gpd.GeoDataFrame(pd.concat([zones,zones_city], ignore_index=True), crs=zones.crs)
    elif os.path.isfile(directory_pc_per_city + file):
        zones_city = gpd.read_file(directory_pc_per_city + file, layer = 'zone')
        zones = gpd.GeoDataFrame(pd.concat([zones,zones_city], ignore_index=True), crs=zones.crs)

In [5]:
exposure_by_PC = pd.read_csv('../data/results/exposure/exposure_by_spatial_unit.csv')

In [6]:
test = zones.merge(exposure_by_PC, on = 'id_unit', how = 'outer')

In [7]:
std_per_municipality = test[['gemeente','expos_NW']].groupby(by = 'gemeente').std()
std_per_municipality = std_per_municipality.reset_index()
std_per_municipality = std_per_municipality.rename(columns = {'expos_NW':'expos_std'})

In [8]:
test = test.merge(std_per_municipality,on ='gemeente')

In [9]:
test.to_file('../data/results/exposure_geo.gpkg')

/home/lulu/.pyenv/versions/3.9.7/envs/study_1_venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
